# 02-Advanced: Data analysis
For these exercises we are going to do some text and more advanced analysis.

## Text data
Often we will need to work with unstructured data like text, images, or audio. To use unstructured data in analyis, we will often need to convert it into something more useable, something more "quantified".

Let's start easy by loading a `.csv` file containing some text data we plan to analyse. Run the code below to load `reviews_sample.csv`.

In [ ]:
import pandas as pd
df_reviews = pd.read_csv('../data/reviews_sample.csv')
df_reviews.head()

### Exercise-01: Cleaning text
We want to analyse the `comments` column, but we first need to remove rows with `nan` values in the `comments` column. Use what you learned in `01-Basic.ipynb` to do this below.

In [ ]:
# (SOLUTION)


### Exercise-02: Analysing text
Now the text data is cleaned, we are going to convert the text to quantities of interest. A common way to do this is to estimate the *sentiment* of the text. There are many ways to analyse sentiment, and here we are going to use the [VADER](https://www.nltk.org/_modules/nltk/sentiment/vader.html) sentiment analytics tool which is included in the [NLTK](https://www.nltk.org/) (Natural Language Toolkit) package.

Run the code below to import `nltk`, import the `SentimentIntensityAnalyzer` class from `nltk`, download the `vader_lexicon`, and create an instance of `SentimentIntensityAnalyzer`.

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
scorer = SentimentIntensityAnalyzer()

Now we have loaded our sentiment analysis tool, we can calculate the sentiment of the comments in the Airbnb reviews. To do this, we are going to create a `calculate_sentiment` function and then apply it to a small sample (`N=5`) of rows from `df_reviews`. To focus our attention on the columns `['comments','sentiment']` we are going to show only those in the `head()` of `df_reviews_sample`. Please run the code below and watch the magic happen!

In [ ]:
def calculate_sentiment(comment):
    return(scorer.polarity_scores(comment)['compound'])

N = 5
df_reviews_sample = df_reviews.sample(N)
df_reviews_sample.loc[:,'sentiment'] = df_reviews_sample['comments'].apply(calculate_sentiment)
df_reviews_sample[['comments','sentiment']].head()

Run the above code a few times, to get a feel for how the sentiment analysis tool is working. Is there anything unexpected/interesting?

## Segmented data
Sometimes we want to perform analysis on only a segment on the data. For example, someone might ask *what is the most expensive listing in London?*, and to solve this we would only need to analysis data for listings based in London.

To explore this idea, let's analyse some segments of the `listings_sample.csv` data. Run the code below to load `listings_sample.csv`.

In [ ]:
import pandas as pd
df_listings = pd.read_csv('../data/listings_sample.csv')
df_listings.head()

Let's assume we want to segment the listings by `room_type`. First, let's take a quick look at how many listings of each `room_type` there are in `df_listings`. To do this, we can use the `value_counts()` method. Please run the code below.

In [ ]:
df_listings['room_type'].value_counts().head()

### Exercise-03: Identify luxury homes
You are a business analyst at a company which a business development manager who asks you to identify the 5 most expensive listings which are entire homes/apartments.

To do this, you first need to create a new `price_$` column in `df_listings` like we did in `01-Basic.ipynb`. Please do that using the `format_price` function written below.

In [ ]:
def format_price(price):
    return(float(price.replace('$','').replace(',','')))

# (SOLUTION)


Now we have price values in a format we can use, we can segment the data and run our analysis to meet the manager's request. To do this, we will:

1. Segment `df_listings` to create a new `DataFrame` named `df_entire_home_apt` which contains only listings with `room_type` equal to `Entire home/apt`.
2. Sort the rows of `df_entire_home_apt` by the values in `price_$` from largest (most expensive) to smallest (cheapest).
3. Display the top 5 rows to show columns we think most relevant for the 5 most expensive listings which are entire homes/apartments.

We will do all this by running only the 3 lines of code shown below. Please run the code.

In [ ]:
df_entire_home_apt = df_listings[df_listings['room_type']=='Entire home/apt']
df_entire_home_apt = df_entire_home_apt.sort_values(by=['price_$'],ascending=False)
df_entire_home_apt[['id','name','description','neighbourhood','price_$']].head(5)

### Exercise 04: Identify budget rooms
the same business development manager now asks you to identify the 5 cheapest listings which are private rooms. Using the above exercise as a guide, run an analysis to meet the manager's request.

In [ ]:
# (SOLUTION)


## Advanced request
Well done so far! Let's now take it too the next level! Let's used what we have learned so far to meet the following request:

*Identify the 5 listings with highest positive mean sentiment of their reviews, such that the listings have at least 5 reviews less than 3 years old*

### Exercise-05: Filter review data
First we need to load `reviews_sample.csv` and filter it to only keep reviews with:
1. values of not `nan` in the `comments` column,
2. values less than 3 years old in the `date` column, and
3. values in the `listing_id` column corresponding to listings with at least 5 reviews.

This can all be done by running the following code.

In [ ]:
import pandas as pd
df_reviews = pd.read_csv('../data/reviews_sample.csv')

df_reviews = df_reviews[~df_reviews['comments'].isna()]

df_reviews = df_reviews[df_reviews['date']>'2018-10-05']

listing_counts = df_reviews['listing_id'].value_counts()
valid_listings = listing_counts[listing_counts>=5].index
df_reviews = df_reviews[df_reviews['listing_id'].isin(valid_listings)]

We can do a "sanity check" by running the code below, which should all produce value `0`. Consider how you might use such test to check your code as you develop it.

In [ ]:
print(sum(df_reviews['comments'].isna()))
print(sum(df_reviews['date']<='2018-10-05'))
print(sum(df_reviews['listing_id'].value_counts()<5))

### Exercise-06: Analyse review data
We already have most of the pieces we need. Simply `apply` the `calculate_sentiment` function (in the way we did before) to the `comments` column of the (now filtered) `df_reviews` to create a new column named `sentiment` containing the a sentiment value for each reviews comment. Please write the code below and run.

(Note that we are no running the `calculate_sentiment` over every row in the filtered `df_reviews` so it might take a while longer...)

In [ ]:
# SOLUTION


Next we need to calculate the mean sentiment for each listing. To do this, we use the `groupby` and `agg` methods on the following way to create a new `DataFrame` named `listing_scored` containing each `listings_id` and the `mean` of its `sentiment` scores. Please run the code below.

In [ ]:
listings_scored = df_reviews.groupby('listing_id')['sentiment'].agg(['mean']).reset_index()
listings_scored.head()

Finally, we just need to sort the listings in `listings_scored` by their `mean` value (from high to low) and print the top 5. Easy, you've got this! Please use what you've learned before to do this below.

In [ ]:
# SOLUTION


## Inspect results
Have a play around with the above code. Maybe try to fetch and inspect the reviews of the listings with high and low sentiment socres, and see what types of listings these are.

Or, if you've found this notebook too easy, have a go at `03-Expert.ipynb`.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c6f047c-21a6-4149-814c-b3f60a9bf973' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>